# **Desafio Técnico - Preparação dos Dados**

Americanas - Cientista de Dados Júnior | BEE

Candidato: Rafael Pavan

Linkedin: https://www.linkedin.com/in/engrafaelpavan/

In [1]:
from sklearn.model_selection import train_test_split

import os

import warnings

import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

import numpy as np

warnings.simplefilter("ignore")

dir = os.getcwd()

dados = pd.read_parquet(f"{dir}/dados_brutos/dataset_cdjr.parquet.gzip", engine='pyarrow')

dados.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,target
337,200.0,2,662.28,39.10,-188.55,0.246978,761,0.004548,3.523703,167326,33441.06,0.019804,26.850,0.009198,94.611429,7,0
266,150.0,2,0.00,149.55,-0.45,0.150000,3,0.037975,0.000000,79,78.93,0.000000,0.000,0.000000,0.000000,0,1
236,50.0,1,346.08,30.41,-102.10,2.430952,42,0.004239,3.389618,9907,18858.77,0.018351,25.525,0.095238,86.520000,4,0
274,100.0,2,0.00,43.84,-56.16,0.150968,372,0.005854,0.000000,63544,1164.11,0.000000,0.000,0.000000,0.000000,0,1
208,50.0,1,87.56,-3.05,-94.50,0.412664,229,0.004572,0.926561,50089,1786.26,0.049019,94.500,0.004367,87.560000,1,0


### 1. Separação em Dados de Treino, Validação e Teste

- 85% Treino e Validação

- 15% Teste

In [2]:
X = dados.iloc[:,:-1]

y = dados.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15, random_state=13)

In [3]:
X_train.to_csv('X_train_bruto.csv', index=False)
X_test.to_csv('X_test_bruto.csv', index=False)

y_train.to_csv('y_train_bruto.csv', index=False)
y_test.to_csv('y_test_bruto.csv', index=False)

### 2. Coletando informações dos Outliers

Vamos encontrar os limitantes pelos quais iremos ceifar os outliers:

In [4]:
def encontrar_limites_outliers(df_input):

    df_out = df_input.copy()

    colunas = []

    linf = []

    lsup = []

    for col_name in df_out.columns:

        colunas.append(col_name)

        q1 = df_out[col_name].quantile(0.25)

        q3 = df_out[col_name].quantile(0.75)
        
        iqr = q3-q1 # amplitude interquartil
        
        low  = q1-3*iqr

        linf.append(low)
        
        high = q3+3*iqr

        lsup.append(high)
        
        df_out[col_name][df_out[col_name] > high] = high

        df_out[col_name][df_out[col_name] < low] = low

    data = pd.DataFrame(columns=colunas, data=[linf, lsup], index=['limite_inferior', 'limite_superior'])

    data.to_csv('outliers_limites.csv')

    return df_out


X_train_clamped = encontrar_limites_outliers(X_train)


In [5]:
lim = pd.read_csv('outliers_limites.csv')

lim.head()

,Unnamed: 0,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15
0,limite_inferior,-1300.0,-14.0,-3086.9475,-135.6725,-1872.660,-1.034566,-3295.00,-0.015070,-7.716400,-762449.50,-13799.0475,-0.9645,-201.415000,-0.027799,-384.137500,-24.0
1,limite_superior,1850.0,21.0,4115.9300,179.9225,1316.295,1.783895,4763.75,0.027821,10.288533,1069630.75,19359.6725,1.2860,268.553333,0.037065,512.183333,32.0


In [6]:
def ceifar_treino(dadosteste, limites):
    
    dadosteste_out = dadosteste.copy()
    
    for col_name in dadosteste:
        
        dadosteste_out[col_name][dadosteste_out[col_name]<limites[col_name][0]] = limites[col_name][0]

        dadosteste_out[col_name][dadosteste_out[col_name]>limites[col_name][1]] = limites[col_name][1]
        
    return dadosteste_out


In [7]:
X_test_clamped = ceifar_treino(X_test,lim)

In [8]:
X_train_clamped

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15
468,1850.0,6,1032.35,179.75,-1872.66,1.327248,1446.0,0.001851,0.537906,781204.00,7241.1900,0.142566,268.553333,0.004841,147.478571,7
348,1850.0,2,4115.93,-3.95,-1872.66,0.716808,2594.0,0.002237,3.477541,1069630.75,1679.6000,1.286000,41.320000,0.017348,143.692000,32
494,400.0,8,243.00,-6.53,-406.53,1.436502,283.0,0.001760,0.597742,160839.00,1481.6800,0.164003,268.553333,0.003534,243.000000,1
522,1000.0,10,4115.93,-5.20,-1026.50,0.563517,1757.0,0.009630,5.596000,182460.00,19149.7300,0.289330,141.442857,0.003984,512.183333,7
434,250.0,5,89.90,-37.17,-255.76,1.783895,138.0,0.002234,0.351501,61784.00,656.3000,0.136980,255.760000,0.007246,89.900000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,150.0,1,79.90,-0.75,-150.75,0.297925,506.0,0.005901,0.530017,85752.00,134.8200,0.592642,150.750000,0.001976,79.900000,1
564,750.0,15,4115.93,11.65,-770.00,0.500000,1495.0,0.011422,7.446020,130892.00,19359.6725,0.082029,41.527778,0.012040,309.216667,18
225,200.0,1,120.25,25.00,-259.95,0.150000,1669.0,0.025870,0.480328,64515.00,1441.7100,0.083408,125.175000,0.001198,60.125000,2
340,100.0,2,1169.22,37.45,-62.55,0.150000,417.0,0.010087,10.288533,41339.00,10846.4600,0.107797,7.818750,0.019185,146.152500,8


### 3. Transformação Yeo-Johnson e MinMaxScaler

Vamos amenizar a assimetria das distribuições e em seguida colocar os dados no range [0,1], o que pode ser benéfico para modelos de ML baseados no cálculo de distância (como KNN)

In [9]:
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline

preprocessing = Pipeline(steps=[
    ('PT', PowerTransformer(method='yeo-johnson')),
    ('MS', MinMaxScaler())])


X_train_processado = X_train_clamped.copy()

X_test_processado = X_test_clamped.copy()

X_train_processado.iloc[:,:] = preprocessing.fit_transform(X_train_clamped.iloc[:,:])
X_test_processado.iloc[:,:] = preprocessing.transform(X_test_clamped.iloc[:,:])

### 4. Retirar Feature4

Vamos retirar a feature4 pois constatamos que apresenta alta correlação com a feature0:


In [10]:
X_train_processado = X_train_processado.drop(columns=['feature4'], axis=1)
X_test_processado = X_test_processado.drop(columns=['feature4'], axis=1)

X_train_processado.to_csv('X_train_processado.csv', index=False)
X_test_processado.to_csv('X_test_processado.csv', index=False)

y_train.to_csv('y_train_processado.csv', index=False)
y_test.to_csv('y_test_processado.csv', index=False)

In [11]:
X_train_processado.head()

,feature0,feature1,feature2,feature3,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15
468,1.000000,0.715576,0.791317,0.999615,0.943078,0.753707,0.151452,0.290507,0.932592,0.808591,0.409706,1.000000,0.400225,0.683837,0.729683
348,1.000000,0.446092,1.000000,0.556412,0.744269,0.868416,0.195622,0.767166,1.000000,0.578253,1.000000,0.639748,0.848982,0.678102,1.000000
494,0.633191,0.785142,0.593843,0.547072,0.960574,0.486267,0.140637,0.312993,0.650775,0.561082,0.452362,1.000000,0.311032,0.800508,0.302636
522,0.836326,0.837637,1.000000,0.551921,0.638637,0.790727,0.717870,0.879767,0.670152,0.997694,0.640903,0.872351,0.343134,1.000000,0.729683
434,0.545123,0.670695,0.469756,0.426385,1.000000,0.389643,0.195272,0.210656,0.518628,0.458242,0.397958,0.990134,0.536004,0.580132,0.302636


In [12]:
X_train_processado.describe()

,feature0,feature1,feature2,feature3,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15
count,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000
mean,0.491589,0.486938,0.457511,0.646482,0.393251,0.565641,0.508711,0.395688,0.568421,0.577903,0.353116,0.435541,0.344998,0.401633,0.412003
std,0.274214,0.293767,0.376263,0.160395,0.319222,0.220555,0.263130,0.360148,0.230071,0.246907,0.353449,0.367023,0.344278,0.317959,0.369194
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.309646,0.295513,0.000000,0.568583,0.061298,0.407582,0.307302,0.000000,0.404953,0.405646,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.460106,0.446092,0.549651,0.621074,0.340474,0.553742,0.451918,0.419283,0.564144,0.557361,0.277520,0.527672,0.282132,0.541560,0.449104
75%,0.678624,0.715576,0.790847,0.686820,0.648758,0.735380,0.709928,0.689046,0.742373,0.749486,0.676588,0.729451,0.626734,0.653072,0.757473
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
X_test_processado.head()

,feature0,feature1,feature2,feature3,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15
597,0.903194,1.000000,0.937475,0.572675,0.780412,0.779004,0.570213,0.637949,0.718343,1.000000,0.220199,0.681505,0.804814,0.621621,0.963179
194,0.309646,0.295513,0.540473,0.599851,0.485859,0.402816,0.300016,0.685320,0.485923,0.532338,0.391503,0.717685,0.499339,0.701878,0.302636
55,0.000000,0.000000,0.548912,0.749501,0.349211,0.444709,0.745204,0.690179,0.393695,0.317448,0.968802,0.726813,0.395168,0.717082,0.302636
148,0.309646,0.295513,0.000000,0.564113,0.821615,0.289599,0.237365,0.000000,0.403981,0.224922,0.000000,0.000000,0.000000,0.000000,0.000000
544,0.810633,0.879376,0.715603,0.623191,0.758924,0.717084,0.446302,0.343559,0.714766,0.677028,0.487824,0.800936,0.553037,0.523827,0.781401


In [14]:
X_test_processado.describe()

,feature0,feature1,feature2,feature3,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15
count,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,0.413324,0.394203,0.434775,0.646007,0.354082,0.503063,0.559248,0.430328,0.501322,0.547763,0.300617,0.353928,0.382664,0.368161,0.401657
std,0.288835,0.297994,0.384294,0.123214,0.303798,0.246120,0.265017,0.402919,0.238873,0.279150,0.337283,0.328409,0.382690,0.309294,0.384956
min,0.000000,0.000000,0.000000,0.264901,0.000000,0.000000,0.095659,0.000000,-0.030311,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.309646,0.295513,0.000000,0.568762,0.073105,0.305568,0.327536,0.000000,0.329402,0.320064,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.399768,0.295513,0.513788,0.610575,0.323990,0.455035,0.535862,0.427585,0.487734,0.532357,0.218778,0.357081,0.365431,0.528101,0.302636
75%,0.626680,0.544051,0.768584,0.686169,0.569884,0.713165,0.782715,0.782545,0.693522,0.765109,0.535386,0.688375,0.755324,0.609832,0.729683
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.804179,1.000000


In [15]:
y_train

468    1
348    1
494    1
522    0
434    0
      ..
206    1
564    1
225    0
340    1
433    1
Name: target, Length: 396, dtype: int64

In [16]:
y_test

597    0
194    1
55     0
148    1
544    0
      ..
599    1
80     1
488    0
391    0
127    0
Name: target, Length: 70, dtype: int64

In [17]:
import pickle 

with open('pipeline.pkl', 'wb') as pickle_file:
    
    pickle.dump(preprocessing, pickle_file)